In [ ]:
# Install the required libraries
!pip install -U transformers datasets trl peft accelerate bitsandbytes

In [ ]:
# Important imports for model training and data handling
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
import datasets
import pandas as pd
import numpy as np

In [ ]:
# Upload the dataset from the local machine
from google.colab import files
uploaded = files.upload()

In [ ]:
# Load the dataset into a pandas DataFrame and display the first few rows
df = pd.read_csv("Barbie_ChatGPT_genAI.csv")
df.head()

In [ ]:
# Format the dataset by creating a new column for training with specific instruction-response format
df["formatted_instruction"] = df.apply(lambda x: f"### Instruction:\n{x['text']}\n\n### Response:\n{x['labels']}", axis=1)

In [ ]:
# Display the first few rows of the DataFrame with the formatted instructions
df.head()

In [ ]:
# Login to Hugging Face Hub (requires user token)
from huggingface_hub import login
login(token="")

In [ ]:
# Display the GPU information if available
!nvidia-smi

In [ ]:
# Initialize the base model and tokenizer
base_model = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

In [ ]:
# Calculate token lengths for formatted instructions and add to DataFrame
df["formatted_instruction_tok_len"] = df["formatted_instruction"].apply(lambda x:len(tokenizer.encode(x)))


In [ ]:
# Plot the distribution of token lengths to visualize
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=df["formatted_instruction_tok_len"])

plt.xlabel("formatted_instruction_tok_len")
plt.title("token length")
plt.show()

In [ ]:
# Filter dataset to keep instructions with token lengths less than or equal to 128
df = df[df["formatted_instruction_tok_len"]<=128]
df.shape

In [ ]:
# Convert the filtered DataFrame to a Hugging Face Dataset
dataset = datasets.Dataset.from_pandas(df)

In [ ]:
# Define configuration for model quantization to reduce memory usage
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)


In [ ]:
import time

In [ ]:
# Load the model with 4-bit quantization configuration
start = time.time()
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1
print("time for model load: {} seconds".format(time.time()-start))

In [ ]:
# Setup the text-generation pipeline with the loaded model and tokenizer
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
prompt = "###Instruction:\nPlease tell me in brief about the cast of the movie called \"Barbie\"\n\n###Response\n:"
gen_text = pipe(prompt)
print(gen_text[0]['generated_text'][len(prompt):])

In [ ]:
# Configure tokenizer to pad tokens and set padding side
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Define parameters for Parameter Efficient Fine-Tuning (PEFT)
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# Define training arguments for model fine-tuning
training_params = TrainingArguments(
    output_dir="./",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
# Initialize the trainer for fine-tuning the model
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="formatted_instruction",
    max_seq_length=128,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

In [ ]:
# Start model training and measure the time taken
start = time.time()
output = trainer.train()
print("Time taken: ", time.time()-start)

In [ ]:
# Generate text using the fine-tuned model to evaluate performance
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
prompt = "###Instruction:\nPlease tell me in brief about the movie called \"Barbie\"\n\n###Response\n:"
start = time.time()
gen_text = pipe(prompt)
print(gen_text[0]['generated_text'][len(prompt):])
print("Time taken: ", time.time()-start)

In [ ]:
tokens = tokenizer.tokenize(text)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

# Calculate the number of words and tokens
num_words = len(text.split())
num_tokens = len(tokens)

# Calculate the ratio
words_per_token = num_words / num_tokens
tokens_per_word = num_tokens / num_words